In [1]:
!pip install -q sentence-transformers scikit-learn spacy nltk
!python -m nltk.downloader punkt
!python -m spacy download en_core_web_sm



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nandhinirajasekaran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import spacy
import nltk
nltk.download('punkt')

nlp = spacy.load("en_core_web_sm")
embedder = SentenceTransformer("all-MiniLM-L6-v2")


/Users/nandhinirajasekaran/Desktop/LLM/RAFT/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nandhinirajasekaran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import json

with open("raft_qa_dataset_semantic.json") as f:
    qa_dataset = json.load(f)

chunks = list({(qa['source_file'], qa['plan_type']): [] for qa in qa_dataset})  # Placeholder for grouping by doc
questions = [qa['question'] for qa in qa_dataset]
answers = [qa['answer'] for qa in qa_dataset]
all_qa_texts = [qa['question'] + " " + qa['answer'] for qa in qa_dataset]


In [4]:
from nltk.tokenize import sent_tokenize

import spacy
nlp = spacy.load("en_core_web_sm")

def spacy_sent_tokenize(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]

def chunk_coherence_score(chunks, model):
    scores = []
    for chunk in chunks:
        sentences = spacy_sent_tokenize(chunk)
        if len(sentences) < 2:
            scores.append(1.0)
            continue
        embeddings = model.encode(sentences)
        sims = [cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
                for i in range(len(embeddings)) for j in range(i+1, len(embeddings))]
        scores.append(np.mean(sims))
    return np.mean(scores)


In [5]:
def qa_precision_score(qa_dataset, model):
    correct = 0
    total = len(qa_dataset)
    for qa in qa_dataset:
        question = qa['question']
        answer = qa['answer']
        chunk_text = answer + " " + question  # Approximate original chunk

        q_emb = model.encode([question])[0]
        c_emb = model.encode([chunk_text])[0]
        sim = cosine_similarity([q_emb], [c_emb])[0][0]
        if sim > 0.65:
            correct += 1
    return correct / total


In [6]:
def redundancy_score(questions, model):
    embeddings = model.encode(questions)
    redundancy = 0
    n = len(embeddings)
    for i in range(n):
        for j in range(i+1, n):
            sim = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
            if sim > 0.9:
                redundancy += 1
    return redundancy / n


In [7]:
def extract_nouns(text):
    doc = nlp(text)
    return {token.lemma_ for token in doc if token.pos_ in ["NOUN", "PROPN"]}

def coverage_score(qa_dataset, full_text):
    text_nouns = extract_nouns(full_text)
    qa_nouns = set()
    for qa in qa_dataset:
        qa_nouns.update(extract_nouns(qa['question'] + " " + qa['answer']))
    if not text_nouns:
        return 0
    return len(qa_nouns & text_nouns) / len(text_nouns)


In [10]:
# Assuming you have raw document text available
with open("datastore/phi-basic.pdf", "rb") as f:
    from pdfplumber import open as pdfopen
    with pdfopen(f) as pdf:
        full_text = "\n".join(page.extract_text() for page in pdf.pages)

coherence = chunk_coherence_score(answers, embedder)
precision = qa_precision_score(qa_dataset, embedder)
redundancy = redundancy_score(questions, embedder)
coverage = coverage_score(qa_dataset, full_text)

print("\n📊 Evaluation Metrics:")
print(f"🧠 Chunk Coherence : {coherence:.3f}")
print(f"✅ QA Precision    : {precision:.3f}")
print(f"🔁 Redundancy      : {redundancy:.3f}")
print(f"📚 Coverage        : {coverage:.3f}")



📊 Evaluation Metrics:
🧠 Chunk Coherence : 1.000
✅ QA Precision    : 0.988
🔁 Redundancy      : 0.814
📚 Coverage        : 0.329


In [2]:
import json

with open("raft_qa_dataset_agentic.json") as f:
    qa_dataset = json.load(f)

chunks = list({(qa['source_file'], qa['plan_type']): [] for qa in qa_dataset})  # Placeholder for grouping by doc
questions = [qa['question'] for qa in qa_dataset]
answers = [qa['answer'] for qa in qa_dataset]
all_qa_texts = [qa['question'] + " " + qa['answer'] for qa in qa_dataset]


In [10]:
# Assuming you have raw document text available
with open("datastore/phi-basic.pdf", "rb") as f:
    from pdfplumber import open as pdfopen
    with pdfopen(f) as pdf:
        full_text = "\n".join(page.extract_text() for page in pdf.pages)

coherence = chunk_coherence_score(answers, embedder)
precision = qa_precision_score(qa_dataset, embedder)
redundancy = redundancy_score(questions, embedder)
coverage = coverage_score(qa_dataset, full_text)

print("\n📊 Evaluation Metrics:")
print(f"🧠 Chunk Coherence : {coherence:.3f}")
print(f"✅ QA Precision    : {precision:.3f}")
print(f"🔁 Redundancy      : {redundancy:.3f}")
print(f"📚 Coverage        : {coverage:.3f}")



📊 Evaluation Metrics:
🧠 Chunk Coherence : 0.956
✅ QA Precision    : 0.908
🔁 Redundancy      : 0.508
📚 Coverage        : 0.285


In [11]:
import json

with open("raft_qa_dataset_llm_chunked.json") as f:
    qa_dataset = json.load(f)

chunks = list({(qa['source_file'], qa['plan_type']): [] for qa in qa_dataset})  # Placeholder for grouping by doc
questions = [qa['question'] for qa in qa_dataset]
answers = [qa['answer'] for qa in qa_dataset]
all_qa_texts = [qa['question'] + " " + qa['answer'] for qa in qa_dataset]

# Assuming you have raw document text available
with open("datastore/phi-basic.pdf", "rb") as f:
    from pdfplumber import open as pdfopen
    with pdfopen(f) as pdf:
        full_text = "\n".join(page.extract_text() for page in pdf.pages)

coherence = chunk_coherence_score(answers, embedder)
precision = qa_precision_score(qa_dataset, embedder)
redundancy = redundancy_score(questions, embedder)
coverage = coverage_score(qa_dataset, full_text)

print("\n📊 Evaluation Metrics:")
print(f"🧠 Chunk Coherence : {coherence:.3f}")
print(f"✅ QA Precision    : {precision:.3f}")
print(f"🔁 Redundancy      : {redundancy:.3f}")
print(f"📚 Coverage        : {coverage:.3f}")



📊 Evaluation Metrics:
🧠 Chunk Coherence : 0.921
✅ QA Precision    : 0.902
🔁 Redundancy      : 0.344
📚 Coverage        : 0.257


In [ ]:
import json

with open("raft_qa_dataset_late_chunking.json") as f:
    qa_dataset = json.load(f)

chunks = list({(qa['source_file'], qa['plan_type']): [] for qa in qa_dataset})  # Placeholder for grouping by doc
questions = [qa['question'] for qa in qa_dataset]
answers = [qa['answer'] for qa in qa_dataset]
all_qa_texts = [qa['question'] + " " + qa['answer'] for qa in qa_dataset]

# Assuming you have raw document text available
with open("datastore/phi-basic.pdf", "rb") as f:
    from pdfplumber import open as pdfopen
    with pdfopen(f) as pdf:
        full_text = "\n".join(page.extract_text() for page in pdf.pages)

coherence = chunk_coherence_score(answers, embedder)
precision = qa_precision_score(qa_dataset, embedder)
redundancy = redundancy_score(questions, embedder)
coverage = coverage_score(qa_dataset, full_text)

print("\n📊 Evaluation Metrics:")
print(f"🧠 Chunk Coherence : {coherence:.3f}")
print(f"✅ QA Precision    : {precision:.3f}")
print(f"🔁 Redundancy      : {redundancy:.3f}")
print(f"📚 Coverage        : {coverage:.3f}")



📊 Evaluation Metrics:
🧠 Chunk Coherence : 1.000
✅ QA Precision    : 0.727
🔁 Redundancy      : 0.818
📚 Coverage        : 0.076


In [13]:
import json

with open("raft_qa_dataset_context_chunking.json") as f:
    qa_dataset = json.load(f)

chunks = list({(qa['source_file'], qa['plan_type']): [] for qa in qa_dataset})  # Placeholder for grouping by doc
questions = [qa['question'] for qa in qa_dataset]
answers = [qa['answer'] for qa in qa_dataset]
all_qa_texts = [qa['question'] + " " + qa['answer'] for qa in qa_dataset]

# Assuming you have raw document text available
with open("datastore/phi-basic.pdf", "rb") as f:
    from pdfplumber import open as pdfopen
    with pdfopen(f) as pdf:
        full_text = "\n".join(page.extract_text() for page in pdf.pages)

coherence = chunk_coherence_score(answers, embedder)
precision = qa_precision_score(qa_dataset, embedder)
redundancy = redundancy_score(questions, embedder)
coverage = coverage_score(qa_dataset, full_text)

print("\n📊 Evaluation Metrics:")
print(f"🧠 Chunk Coherence : {coherence:.3f}")
print(f"✅ QA Precision    : {precision:.3f}")
print(f"🔁 Redundancy      : {redundancy:.3f}")
print(f"📚 Coverage        : {coverage:.3f}")



📊 Evaluation Metrics:
🧠 Chunk Coherence : 0.873
✅ QA Precision    : 0.868
🔁 Redundancy      : 0.824
📚 Coverage        : 0.389


In [14]:
import json

with open("raft_qa_dataset_hybrid.json") as f:
    qa_dataset = json.load(f)

chunks = list({(qa['source_file'], qa['plan_type']): [] for qa in qa_dataset})  # Placeholder for grouping by doc
questions = [qa['question'] for qa in qa_dataset]
answers = [qa['answer'] for qa in qa_dataset]
all_qa_texts = [qa['question'] + " " + qa['answer'] for qa in qa_dataset]

# Assuming you have raw document text available
with open("datastore/phi-basic.pdf", "rb") as f:
    from pdfplumber import open as pdfopen
    with pdfopen(f) as pdf:
        full_text = "\n".join(page.extract_text() for page in pdf.pages)

coherence = chunk_coherence_score(answers, embedder)
precision = qa_precision_score(qa_dataset, embedder)
redundancy = redundancy_score(questions, embedder)
coverage = coverage_score(qa_dataset, full_text)

print("\n📊 Evaluation Metrics:")
print(f"🧠 Chunk Coherence : {coherence:.3f}")
print(f"✅ QA Precision    : {precision:.3f}")
print(f"🔁 Redundancy      : {redundancy:.3f}")
print(f"📚 Coverage        : {coverage:.3f}")



📊 Evaluation Metrics:
🧠 Chunk Coherence : 1.000
✅ QA Precision    : 0.968
🔁 Redundancy      : 6.690
📚 Coverage        : 0.759
